# Обработка данных, полученных в китайском эксперименте с квантовыми точками

### Необходимые для расчета величины:
- [X] Trace для КТ
- [X] Бинирование данных
- [ ] Данные для диаграммы "ON/OFF"
- [ ] Корреляционная функция
- [ ] Данные для диаграммы FLID. 1000 фотонов
- [ ] Данные для диаграммы FLID. Как в лебеге

### Необходимые в качестве результата графики:
- [X] Trace plot
- [ ] Intensity histogram
- [ ] "ON/OFF" diagram
- [ ] Correlate function plot
- [ ] FLID, 1000
- [ ] FLID, Lebeg

## Вспомогательные по отношению к обработке данных методы и структуры данных

In [ ]:
using BenchmarkTools
using CairoMakie
using Statistics

In [ ]:
struct ParsedData 
    channel::Vector
    delays::Vector
    arrivals::Vector
end

struct BinnedData
    intensity::Vector
    index_min::Vector
    index_max::Vector
end

## Подготовка данных
### Парсинг данных из исходников

Исходные файлы приходят в виде бинарных слов, состоящих из 32-битных слов. Данные начинаются со слова **EB 90 B7 13**, а заканчиваются словом **EE EE DD DD**. 

Структура 32-битного слова следующая:
| control_bit (1 bit) | channel (6 bit) | delays (15 bit) | SPSN (10 bit) |
| ---- | ---- | ---- | ---- |
| 0: в слове информация о фотонах  | Номер канала | Задержка относительно синхроимпульса | Номер синхроимпульса |
| 1: в слове информация о переполнении SPSN | 111111 | All 0 | Количество переполнений |

In [ ]:
DataReadFromFile(path::String) = hton.(reinterpret(UInt32, read(path)));

In [ ]:
function ParsingData(source_data::Vector{UInt32})
    # Разбор 32-битных слов на компоненты
    control_bit = UInt8.((source_data .>> 31) .& 0x01)
    channel     = UInt8.((source_data .>> 25) .& 0x3F)
    delays      = UInt64.((source_data .>> 10) .& 0x7FFF)
    impulse_num = UInt64.(source_data .& 0x3FF)

    # Создание массива для корректировки переполнений
    overflow_idxs = findall(==(1), control_bit)
    overflow_diff = Statistics.diff(overflow_idxs) .- 1
    overflow_vec  = collect(0 : length(overflow_diff)-1)

    overflow = Vector{UInt64}(undef, sum(overflow_diff))
    i = 1
    @inbounds for j in eachindex(overflow_vec)
        value   = overflow_vec[j]
        repeats = overflow_diff[j]
        for k in 1 : repeats
            overflow[i] = value
            i += 1
        end
    end
    overflow .*= 1023

    arrivals = impulse_num

    # Удаление битов с control_bit = 1
    mask = ones(Bool, length(control_bit))
    mask[overflow_idxs] .= false

    channel  = channel[mask]
    delays   = delays[mask]
    arrivals = arrivals[mask]
    impulse_num = impulse_num[mask]

    # Корректировка битов переполнения и задержек
    impulse_num .+= overflow
    delays .*= 64

    # Формирование arrivals массива
    time_between = maximum(delays)
    arrivals = (delays .+ impulse_num .* UInt64(time_between)) .* 1e-12
    arrivals .-= arrivals[1]
    delays = delays .* 1e-12

    # Сортировка
    perm = sortperm(arrivals)
    delays = delays[perm]
    channel = channel[perm]
    
    return ParsedData(channel, delays, arrivals)
end;

### Бинирование данных

In [ ]:
function BinningData(arrivals::Vector{Float64}, bin_size::Float64)
    nums_of_bin = UInt32(arrivals[end] ÷ bin_size)
    nums_of_arrivals = length(arrivals)

    intensity = Vector{UInt32}(undef, nums_of_bin)
    index_min = Vector{UInt32}(undef, nums_of_bin)
    index_max = Vector{UInt32}(undef, nums_of_bin)

    count = 0
    for j in 1:nums_of_bin
        if j == 1
            index_min[j] = 1
            count = 1
        else
            index_min[j] = index_max[j-1] + 1
            count = index_min[j]
        end

        while arrivals[count] < j*bin_size
            count += 1
            if count + 1 > nums_of_arrivals
                break
            end
        end

        if count == index_min[j]
            intensity[j] = 0
            index_max[j] = index_min[j] - 1
        else
            index_max[j] = count - 1
            intensity[j] = index_max[j] - index_min[j] + 1
        end
    end

    return BinnedData(intensity, index_min, index_max)
end;

In [ ]:
function ON_OFF_Analyse(intensity::Vector{UInt32}, bin_size::Float64, level::Float64)
    bound = level * maximum(intensity)

    n = length(intensity)
    ON_time = Vector{Float64}(undef, n)
    OFF_time = Vector{Float64}(undef, n)

    ON_or_OFF_flag = true
    time = 0

    for (index, bin_intensity) in enumerate(intensity)
        if bin_intensity < bound
            if ON_or_OFF_flag == true
                ON_time[index] = time
                time = 0
            end
            ON_or_OFF_flag = false
        end
    end
end;

In [ ]:
function CalculateCorrelateFunction(parsed_data::ParsedData, intensity::Vector{UIn32})
    arrivals = parsed_data.arrivals
    delays   = parsed_data.delays
    channel  = parsed_data.channel

    ch1 = arrivals(channel .== 0)
    ch2 = arrivals(channel .== 1)

    println(length(arrivlas))
    ptintln(length(ch1) + length(ch2))
end

### Построение Trace plot

In [ ]:
function PlotterTrace(intensity::Vector{UInt32}, bin_size::Float64)
    bin_size_ms = bin_size*1000
    fig = Figure(size=(1500, 500), fontsize=24)
    ax = Axis(fig[1, 1],
              xlabel="Time, [s]",
              ylabel="Intensity, [counts/$bin_size_ms ms]",
              title="55 QD 622 nm in PPD",
              limits=(0, length(intensity)*bin_size, 0, 1.1*maximum(intensity))
        )
    times = range(0.0; step=bin_size, length=length(intensity)) |> collect

    lines!(ax, times, intensity, color=:black)
    fig        
end

## Тест системы

In [ ]:
data = DataReadFromFile("source_data/tests/1");

In [ ]:
parsed_data = ParsingData(data);

In [ ]:
binned_data = BinningData(parsed_data.arrivals, 0.001);

In [ ]:
PlotterTrace(binned_data.intensity, 0.001)

In [ ]:
function DataProcessing(path::String)
    bin_size = 0.001
    
    source_data = DataReadFromFile(path)
    parsed_data = ParsingData(source_data)
    binned_data = BinningData(parsed_data.arrivals, bin_size)
end

In [ ]:
folder = "source_data"
subfolder = "tests"
file = "1"

path = joinpath(folder, subfolder, file)

DataProcessing(path)